In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import matplotlib.pyplot as plt
import tensorflow as tf
import time, os, gc
import numpy as np
from IPython.display import clear_output, Image, display, HTML
from mpl_toolkits.mplot3d import Axes3D
import shutil
from nnBuilder import *
from nnDreamer import *
from dataProducer import *
from lapgan import *
%matplotlib inline

In [ ]:
folder="MapsData/Gravel"
fixDir("models")
model_folder=fixDir("models/lapgan_2016_11_24")
size=np.array([5760,3840])
scale=1
n=4
m=8
dic={"img_file":folder+"/img%i"%scale,
     "img_shape":list(size//scale)+[1],
     "blur_file":folder+"/blur%i"%scale,
     "blur_shape":list(size//scale)+[1],
     "size":32,
     "batch":32,
     "channels":0,
     "noise":.1
    }
dic["gen_def"]={
    "layer_n":4,
    "windows":[5,5,5,5],
    "widths":[16*n,32*n,32*n,1],
    "batch_norm":True,
    "rand_scale":.005
}
dic["class_def"]={
    "layer_n":4,
    "windows":[5,5,5,1],
    "widths":[16*m,32*m,32*m,1],
    "batch_norm":False,
    "dropouts":[False,True,False,True]
}

sess=SessManager([])
network=LapGAN_top(sess,dic)
sess.start()
need_warmup=False

In [ ]:
network.class_trainer.learnRate=0.001
network.class_trainer.keep_rate=1.
network.class_trainer.l2regMul=1e-8
network.class_trainer.momentum=.9
network.gen_trainer.learnRate=0.000001
network.gen_trainer.keep_rate=1.
network.gen_trainer.l2regMul=1e-8
network.gen_trainer.momentum=.9
if need_warmup:
    print("Warming up")
    network.evaluate(10)
    for i in range(1):
        network.train_class(100)
        network.evaluate(10)
        a=sess.sess.run(network.diff)
        print(a.mean(),a.std(),np.mean([a[i].std() for i in range(a.shape[0])]))
    need_warmup=False
while True:
    network.train(1000,2,dt=.005)
    network.evaluate(10)
    a=sess.sess.run(network.diff)
    print(a.mean(),a.std(),np.mean([a[i].std() for i in range(a.shape[0])]))
    network.compare()
    network.sample()
    network.save(model_folder)


In [ ]:
a=sess.sess.run(network.x)
for i in range(16):
    showarray(a[i,:,:,0])

In [ ]:
a=sess.sess.run(network.generator.y)
for i in range(16):
    showarray(a[i,:,:,0])

In [ ]:
a=sess.sess.run(network.diff)
for i in range(16):
    print(a[i,:,:,0].std())
    showarray(a[i,:,:,0])

In [ ]:
a=sess.sess.run(network.y)
for i in range(16):
    showarray(a[i,:,:,0])

In [ ]:
a=sess.sess.run(network.gen_input.batchProducer.producer.tf_arr)
showarray(a[:500,:500,0])

In [ ]:
a=sess.sess.run(network.class_mixer.layers[1].batchProducer.producer.tf_arr)
showarray(a[:500,:500,0])

In [ ]:
network.class_mixer.layers[1].batchProducer.producer.tf_arr

In [ ]:
a=sess.sess.run(network.class_mixer.y)
for i in range(32):
    print(a[i,:,:,0].std())
    showarray(a[i,:,:,0])

In [ ]:
network.classifier.save()

In [ ]:
network.generator.save()

In [ ]:
showarray(np.load(folder+"/diff%i.npy"%scale)[:500,:500])

In [ ]:
dreamer1=SimpleDreamer(self.classifier.input_mixer.layers[1].get(),self.classifier.network.y,sigmoid=True)

In [ ]:
tf_in=tf.placeholder(tf.float32, shape=[None,None,None,1])
classifier=ConvNet(tf_in,dic["class_def"])
dreamer=SimpleDreamerConv(tf_in,classifier.y,sigmoid=True)
class_saver=NetworkSaver(classifier)
sess.add([classifier,dreamer,class_saver])
class_saver.load(model_folder+"classifier")


In [ ]:
network.load(model_folder)

In [ ]:
diff,y=network.sample()
a=sess.sess.run(classifier.y,{tf_in:y})

In [ ]:
dreamer.setImg(y[0,:,:,0])

In [ ]:
dreamer.step=.1
dreamer.train(100)

In [ ]:
showarray(dreamer.get())

In [ ]:
sess.sess.run(classifier.y,{tf_in:dreamer.get()[None,:,:,None]}).mean()

In [ ]:
plot(dreamer.get()[:80,:80])

In [ ]:
size=500
blur_full=np.load(folder+"/blur%i.npy"%scale)[:size+network.size_diff,:size+network.size_diff]
blur_crop=blur_full[network.size_diff//2:-network.size_diff//2,network.size_diff//2:-network.size_diff//2]
img_full=np.load(folder+"/blur%i.npy"%scale)[:size+network.size_diff,:size+network.size_diff]
img_crop=img_full[network.size_diff//2:-network.size_diff//2,network.size_diff//2:-network.size_diff//2]
tf_in=tf.placeholder(tf.float32, shape=[1,size,size,1])
tf_add=tf.add(tf_in,blur_crop[None,:,:,None])
classifier=ConvNet(tf_add,dic["class_def"])
dreamer=SimpleDreamerConv(tf_in,classifier.y,sigmoid=True)
class_saver=NetworkSaver(classifier)
sess.add([classifier,dreamer,class_saver])
class_saver.load(model_folder+"classifier")

In [ ]:
tf_add.get_shape()

In [ ]:
blur_crop.shape

In [ ]:
tf_in+blur_crop[None,:,:,None]

In [ ]:
dreamer.setImg(diff[0,:,:,0])

In [ ]:
dreamer.step=.1
dreamer.train(100)

In [ ]:
showarray(dreamer.get())
showarray(dreamer.get()+blur_crop)
sess.sess.run(classifier.y,{tf_in:dreamer.get()[None,:,:,None]}).mean()

In [ ]:
diff=sess.sess.run(network.diff,{network.x:blur_full[None,:,:,None]})

In [ ]:
showarray(dreamer.get()[:40,:40])